In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
train_df = pd.read_csv(r"C:\Users\ZEESHAN\Desktop\PAI-Lab\train.csv")
test_df = pd.read_csv(r"C:\Users\ZEESHAN\Desktop\PAI-Lab\test.csv")
encoders = {}
def preprocess_data(df, is_train=True):
    df.fillna(method='ffill', inplace=True)
    df[['Deck', 'Num', 'Side']] = df['Cabin'].fillna("Unknown/0/Unknown").str.split('/', expand=True)
    df.drop(columns=['Name', 'PassengerId', 'Cabin'], inplace=True, errors='ignore')
    categorical_features = df.select_dtypes(include=['object']).columns
    for col in categorical_features:
        if is_train:
            encoders[col] = LabelEncoder()
            df[col] = encoders[col].fit_transform(df[col].astype(str))
        else:
            df[col] = df[col].astype(str).map(lambda x: encoders[col].transform([x])[0] if x in encoders[col].classes_ else -1)

    return df
train_df = preprocess_data(train_df, is_train=True)
test_df = preprocess_data(test_df, is_train=False)
X = train_df.drop(columns=['Transported'])
y = train_df['Transported']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
num_features = X_train.select_dtypes(include=['number']).columns
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_val[num_features] = scaler.transform(X_val[num_features])
test_df[num_features] = scaler.transform(test_df[num_features])

# Train Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate Model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

# Make Predictions
test_predictions = model.predict(test_df)

# Create Submission File
submission = pd.DataFrame({
    "PassengerId": pd.read_csv(r"C:\Users\ZEESHAN\Desktop\PAI-Lab\test.csv")["PassengerId"],
    "Transported": test_predictions
})
submission.to_csv("submission.csv", index=False)
print("Submission file saved successfully!")


C:\Users\ZEESHAN\AppData\Local\Temp\ipykernel_5128\748861679.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\ZEESHAN\AppData\Local\Temp\ipykernel_5128\748861679.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(method='ffill', inplace=True)
C:\Users\ZEESHAN\AppData\Local\Temp\ipykernel_5128\748861679.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\ZEESHAN\AppData\Local\Temp\ipykernel_5128\748861679.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill,

Validation Accuracy: 0.7855
Submission file saved successfully!
